# Rover control (draft/development)

This notebook is for development and testing of the Rover. It is meant to be used with the Rover connected to USB during development.

- The communication with vehicle using [The Simple Communication Protocol for Embedded Applications](https://github.com/nhorro/simpleprotocol).

## Protocol definition

In [1]:
import simpleprotocol as sp
import ctypes
from struct import unpack, pack

In [2]:
def bytearray_to_packet_str(packet_ba: bytearray) -> str:
    return "0x " + "".join(["%02X " % x for x in packet_ba])

### Telecommands

In [3]:
class LedControlCommand(sp.BaseMessage):
    def __init__(self):
        # Header
        self.opcode = 0
        
        # Payload
        self.control_flags = 0

        # see: https://docs.python.org/3/library/struct.html
        self.format = '>BI'
        sp.BaseMessage.__init__(self)

In [4]:
class ControlMotorManualCommand(sp.BaseMessage):
    def __init__(self):
        # Header
        self.opcode = 1
        
        # Payload
        self.MotorControlFlags = 0
        self.MotorAThrottle = 0.
        self.MotorBThrottle = 0.

        # see: https://docs.python.org/3/library/struct.html
        self.format = '>BIff'
        sp.BaseMessage.__init__(self)

In [5]:
class ControlMotorAutoCommand(sp.BaseMessage):
    def __init__(self):
        # Header
        self.opcode = 2
        
        # Payload
        self.MotorControlFlags = 0
        self.MotorASpeed = 0.
        self.MotorBSpeed = 0.

        # see: https://docs.python.org/3/library/struct.html
        self.format = '>BIff'
        sp.BaseMessage.__init__(self)

In [6]:
class SetMotorControlModeCommand(sp.BaseMessage):
    def __init__(self):
        # Header
        self.opcode = 3
        
        # Payload
        self.MotorControlModeFlags = 0

        # see: https://docs.python.org/3/library/struct.html
        self.format = '>BI'
        sp.BaseMessage.__init__(self)

### Telemetry

In [7]:
class GeneralTelemetry(sp.BaseMessage):
    def __init__(self):        
        
        self.TelemetryCycle = 0
        self.ReceivedPackets = 0
        self.LedControlState = 0
        self.LastCommandOpcode = 0
        self.LastCommandResult = 0
        self.LasOsResult1 = 0
        self.LasOsResult2 = 0

        # see: https://docs.python.org/3/library/struct.html
        self.format = '>IIIIIii'
        sp.BaseMessage.__init__(self)
    
    def get_field_values(self):        
        result = {f: getattr(self, f) for f in self.fields }
        return result

In [8]:
class MotorControlTelemetry(sp.BaseMessage):
    def __init__(self):        
        
        self.Throttle1 = 0
        self.Throttle2 = 0
        
        self.Tachometer1 = 0
        self.Tachometer2 = 0
        self.Tachometer3 = 0
        self.Tachometer4 = 0
        
        self.MeasuredSpeed1 = 0
        self.MeasuredSpeed2 = 0
        self.MeasuredSpeed3 = 0
        self.MeasuredSpeed4 = 0
        
        self.SetpointSpeed1 = 0
        self.SetpointSpeed2 = 0
        
        self.StatusFlags = 0

        # see: https://docs.python.org/3/library/struct.html
        self.format = '>ffIIIIffffffI'
        sp.BaseMessage.__init__(self)
    
    def get_field_values(self):        
        result = {f: getattr(self, f) for f in self.fields }
        return result

## Serial connection

In [9]:
from serialif import SerialIF
from struct import unpack

class GroundControlSerialIF(SerialIF):
    """ 
    """
    def __init__(self, port, baudrate,debug=False):
        SerialIF.__init__(self, port, baudrate, self._on_data)
        self.debug = debug
        self.decoder = sp.Decoder(
            user_handler_fn=self._handle_packet, 
            user_error_handler_fn=self._handle_error)
        self.encoder = sp.Encoder()
        self.start_listening()
        
        self.telemetry_report_handlers = {
           0: self._handle_general_telemetry,
           1: self._handle_motor_control_telemetry
        }
        
        self.last_general_telemetry = None
        self.last_motor_control_telemetry = None
        self.last_motor_control_telemetry_raw = None
        
    def print_telemetry(self):
        print("General Telemetry")
        for k,v in self.last_general_telemetry.items():
            print("   {}: 0x{:08x}".format(k,v))
        print("Motor Telemetry")
        for k,v in self.last_motor_control_telemetry.items():
            print("   {}: {}".format(k,v))

    def _handle_error(self, ec):
        print("Some error:", ec)

    def shutdown(self):
        self.stop_listening()
        
    def _on_data(self, c):
        self.decoder.feed(ord(c))
        
    def _send_command(self,cmd):
        self.encoder.encode(cmd.to_bytes())
        self.send_data(self.encoder.get_packet_bytes())
        #if self.debug:
        print("Sent: {} ({} bytes)".format(
            bytearray_to_packet_str(self.encoder.get_packet_bytes()),
            len(self.encoder.get_packet_bytes())        
        ))
                        
    def _handle_packet(self, payload):
        #print("Good packet!. Payload:", payload)        
        if self.debug:
            print("Received payload: {}".format(bytearray_to_packet_str(payload)))
            
        report_type = payload[0]
        if report_type in self.telemetry_report_handlers:
            self.telemetry_report_handlers[report_type](payload[1:])
        else:
            print("Unknown report type: 0x{:02x}".format(report_type))
        
    # =====================================================================
    # Commands here
    # =====================================================================
    
    # Commands    
    def control_led(self,led_state):
        cmd = LedControlCommand()        
        cmd.control_flags = led_state
        self._send_command(cmd)
        
    def set_motor_throttles(self,a,b,flags):
        cmd = ControlMotorManualCommand()        
        cmd.MotorControlFlags = flags
        cmd.MotorAThrottle = a
        cmd.MotorBThrottle = b
        self._send_command(cmd)
        
    def set_motor_speeds(self,a,b,flags):
        cmd = ControlMotorAutoCommand()        
        cmd.MotorControlFlags = flags
        cmd.MotorASpeed = a
        cmd.MotorBSpeed = b
        self._send_command(cmd)
        
    def set_motor_control_mode(self,flags):
        cmd = SetMotorControlModeCommand()        
        cmd.MotorControlModeFlags = flags        
        self._send_command(cmd)
        
    # =====================================================================
    # Telemetry report handling here
    # =====================================================================            
            
    def _handle_general_telemetry(self,payload):
        tm = GeneralTelemetry()
        tm.from_bytes(payload)
        self.last_general_telemetry = tm.get_field_values()
        
    def _handle_motor_control_telemetry(self, payload):
        tm = MotorControlTelemetry()
        tm.from_bytes(payload)
        self.last_motor_control_telemetry = tm.get_field_values() 
        self.last_motor_control_telemetry_raw = payload

## Vehicle operation

### Connection to vehicle

In [11]:
vehicle = GroundControlSerialIF("/dev/ttyACM0", 115200,debug=False)

Spurius character:  32


### Sending commands

In [ ]:
vehicle.control_led(0x00000001)

In [29]:
vehicle.set_motor_throttles(12.0,15.0,0x3)

Sent: 0x 40 3C 0D 01 00 00 00 03 41 40 00 00 41 70 00 00 AD 0A  (18 bytes)


In [32]:
vehicle.set_motor_speeds(1.,2.,0x3)

Sent: 0x 40 3C 0D 02 00 00 00 03 3F 80 00 00 40 00 00 00 BF 0A  (18 bytes)


## Checking received telemetry

In [36]:
vehicle.print_telemetry()

General Telemetry
   TelemetryCycle: 0x00000010
   ReceivedPackets: 0x00000001
   LedControlState: 0x00000000
   LastCommandOpcode: 0x00000002
   LastCommandResult: 0x00000001
   LasOsResult1: 0x00000000
   LasOsResult2: 0x00000000
Motor Telemetry
   Throttle1: 0.0
   Throttle2: 0.0
   Tachometer1: 0
   Tachometer2: 0
   Tachometer3: 0
   Tachometer4: 0
   MeasuredSpeed1: 0.0
   MeasuredSpeed2: 0.0
   MeasuredSpeed3: 0.0
   MeasuredSpeed4: 0.0
   SetpointSpeed1: 1.0
   SetpointSpeed2: 2.0
   StatusFlags: 0


## Sending floats is not working

In [ ]:
bytearray_to_packet_str(vehicle.last_motor_control_telemetry_raw)

~~~
00 00 00 00 
00 00 00 00 

00 00 00 00 
00 00 00 00 
00 00 00 00 
00 00 00 00 

00 00 06 C3  
B5 15 69 41 
1B AF 13 42 
A4 E0 7F 44 

00 00 00 00 
00 00 00 00 

00 00 00 00

Motor Telemetry
   Throttle1: 0.0
   Throttle2: 0.0
   Tachometer1: 0
   Tachometer2: 0
   Tachometer3: 0
   Tachometer4: 0
   MeasuredSpeed1: 2.4256476417462583e-42
   MeasuredSpeed2: -5.565999003920297e-07
   MeasuredSpeed3: 2.8963766508996735e-22
   MeasuredSpeed4: -9.736011100955702e-17
   SetpointSpeed1: 0.0
   SetpointSpeed2: 0.0
   StatusFlags: 0

~~~

## Teardown

In [ ]:
vehicle.shutdown()

# Work in progress

<hr/>